# Chapter 5. 오차역전파법 (Backpropagation)  

* 수치 미분의 장점과 단점
    * 장점 - 단순하고 구현이 간단.
    * 단점 - 시간이 오래 걸림.
        * 가중치 매개변수의 기울기를 효율적으로 계산하기 위해 Backpropagation 도입.


* Backpropagation의 이해
    * 수식 - 설명 안함.
    * 계산 그래프(시각적) - CS231n 참고

## 5.1 계산 그래프 (Computational Graph)  

* 계산 그래프 - 계산 과정을 그래프로 표현. (TensorFlow와 비슷한 접근).
    * 노드(Node) - 원(O)으로 표현 - 연산
    * 에지(Edge) - 화살표 - 연산의 흐름

### 5.1.1 계산 그래프로 풀다  

*문제 1 : 현빈 군은 슈퍼에서 1개에 100원인 사과 2개를 샀습니다. 이때 지불금액을 구하세요. 단 소비세가 10% 부과됩니다.*  
(문제 1은 곱으로만 표현 가능)

**그림 5-1 삽입**  
**그림 5-1 계산 그래프로 풀어본 문제 1의 답**

그림 5-1은 상수와 연산이 완전히 분리되어 있지 않으므로 상수와 연산자를 완전히 분리하면 그림 5-2와 같이 표현됨.  
(이후에는 그림 5-2와 같은 형식으로 표현)

**그림 5-2 삽입**  
**그림 5-2 계산 그래프로 풀어본 문제 1의 답 : '사과의 개수'와 '소비세'를 변수로 취급해 원 밖에 표기**

*문제 2 : 현빈 군은 사과를 2개, 귤을 3개 샀습니다. 사과는 1개에 100원, 귤은 1개 150원입니다. 소비자세가 10%일 때 지불 금액을 구하세요.*  
(문제 2는 곱과 합으로 표현 가능)

**그림 5-3 삽입**

**그림 5-3 계산 그래프로 풀어본 문제2의 답**

* 계산 그래프를 이용한 문제풀이의 흐름
    1. 계산 그래프를 구성한다.
    2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다.

2와 같이 왼쪽에서 오른쪽으로 진행하는 단계를 순전파 (Forward Propagation).  
반대로 오른쪽에서 왼쪽으로 진행하는 단계를 역전파 (Backpropagation).

### 5.1.2 국소적 계산  

계산 그래프의 특징은 **국소적 계산**을 전파함으로써 최종 결과를 얻는 것.  
**국소적이란** 자신과 직접 관계된 작은 범위.  
(전체와는 상관없이 자신과 관계된 정보만으로 다음 결과를 출력할 수 있음.)

**그림 5-4 삽입**  
**그림 5-4 사과 2개를 포함해 여러 식품을 구입하는 예**

그림 5-4에서 여러 식품을 구입하여 총금액이 4,000원.  
이후 사과를 추가로 더 구매한다면 두 값을 더하는 계산(4,000 + 200 -> 4,200)은 4,000이라는 숫자가 어떻게 계산되어 나왔느냐와 관계가 없음.  
결국 각 노드는 자신과 관련한 계산(그림 5-4에서는 두 숫자의 덧셈) 외에는 신경쓸 사항이 없음.  


위에서 처럼 계산 그래프는 국소적 계산에만 집중하면 됨.  
**간단한 국소적인 계산의 조합으로 복잡한 계산 결과를 얻을 수 있음.**

### 5.1.3 왜 계산 그래프로 푸는가?  

* 계산 그래프의 이점
    * 국소적 계산
        * 각 노드의 단순한 계산으로 문제를 단순화 할 수 있음.
        * 각 노드의 중간 계산 결과를 모두 보관 가능.
    * **역전파를 통해 *미분*을 효율적으로 계산 가능.**


역전파 설명을 위해 문제 1로 다시 돌아감.  
*문제 1 : 현빈 군은 슈퍼에서 1개에 100원인 사과 2개를 샀습니다. 이때 지불금액을 구하세요. 단 소비세가 10% 부과됩니다.*  

*문제 1-1 : 사과 가격이 오르면 최종 금액에 어떤 영향을 끼치는지 알고 싶음.*  
위 문제는 **사과 가격에 대한 지불 금액의 미분**을 구하는 문제임.

사과 값을 $x$, 지불 금액을 $L$이라 했을 때 $\frac{\partial L}{\partial x}$을 구하는 문제.  
(사과 값이 **아주 조금** 올랐을 때 지불 금액이 얼마나 증가하느냐?)

**사과 가격에 대한 지불 금액의 미분**은 역전파를 하면 구할 수 있음.  
그림 5-5는 계산 그래프 상의 역전파를 미분을 통혜서 구해 놓은 결과.(자세한 내용은 뒤에서)  

**그림 5-5 삽입**  
**그림 5-5 역전파에 의한 미분 값의 전달**

역전파는 순전파와는 반대 방향의 화살표(붉은 선)으로 표시.  
이 전파는 **국소적 미분**을 전달하고 그 미분 값을 화살표의 아래에 표시.  
이 예에서 역전파는 오른쪽에서 왼쪽으로 **1 -> 1.1 -> 2.2** 순으로 미분 값을 전달.  
이 결과로부터 **사과 가격에 대한 지불 금액의 미분**값은 2.2라는 것을 알 수 있음.  
사과가 1원 오르면 최종금액은 2.2원 오름.  
(사과 가격이 $x$만큼 오르면 최종금액은 2.2$x$만큼 오름.)  


## 5.2 연쇄법칙 (Chain Rule)  

그림 5-5처럼 오른쪽에서 왼쪽으로 **국소적 미분**을 전달하는 원리는 **연쇄법칙(Chain Rule)**에 따른 것.  
이 절에서 연쇄법칙을 설명하고 그것이 그래프 상의 역전파와 같다는 사실을 확인.

### 5.2.1 계산 그래프의 역전파  

그림 5-6에서 $y=f(x)$라는 계산의 역전파를 표현.  

**그림 5-6 삽입**  
**그림 5-6 계산 그래프의 역전파 : 순방향과는 반대 방향으로 국소적 미분을 곱한다.**  

그림 5-6과 같이 역전파의 계산 절차는 신호 $E$에 노드의 국소적 미분$(\frac{\partial y}{\partial x})$를 곱한 후 다음 노드로 전달하는 것.  
1. **국소적 미분**은 순전파 때의  $y=f(x)$ 계산의 미분을 구한다는 것이며, 이는 $x$에 대한 $y$의 미분$(\frac{\partial y}{\partial x})$을 구한다는 뜻.  
(예 : $y=f(x)=x^2$이라면 $(\frac{\partial y}{\partial x}) = 2x$)
2. 그리고 이 국소적인 미분을 상류에서 전달된 값(이 예에서는 E)에 곱해 앞으로 전달하는 것.  

위의 방식을 따르면 목표로 하는 미분 값을 효율적으로 구할 수 있다는 것이 이 전파의 핵심.

### 5.2.2 연쇄법칙이란?  

**합성 함수** - 여려 함수로 구성된 함수.  

예) $x = {(x + y)}^2$는 식 5.1처럼 두 개의 식으로 구성.  
$$ \mathbf{식 \ 5.1} \\ z = t^2 \\  t = x + y $$

* 연쇄법칙
    * *합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곲으로 나타낼 수 있다.*  

**식 5.1**을 예로 설명, $\frac{\partial z}{\partial x}$ ($x$에 대한 $z$의 미분)은  $\frac{\partial z}{\partial t}$  ($t$에 대한 $z$의 미분)과  $\frac{\partial t}{\partial x}$($x$에 대한 $t$의 미분)의 곱으로 나타낼 수 있음.  
수식으로는 **식 5.2**처럼 표기.  

$$ \mathbf{식 \ 5.2} \\ \frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} $$  

**식 5.2**는 다음과 같이 $\partial t$를 서로 지울 수 있음.

$$\require{cancel} \frac{\partial z}{\partial x} = \frac{\partial z}{\cancel{\partial t}} \frac{\cancel{\partial t}}{\partial x}$$  

연쇄 법칙을 써서 **식 5.2**의 미분 $\frac{\partial z}{\partial x}$를 진행. 먼저 **식 5.1**의 국소적 미분(편미분)을 구함.  
$$ \mathbf{식 \ 5.3} \\ \frac{\partial z}{\partial t} = 2t \\ \frac{\partial t}{\partial x} = 1 $$


### 5.2.3 연쇄법칙과 계산 그래프

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파

### 5.3.2 곱셈 노드의 역전파

### 5.3.3 사과 쇼핑의 예

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

### 5.4.2 덧셈 계층

## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층

### 5.5.2 Sigmoid 계층

## 5.6 Affine/Softmax 계층 구현하기

### 5.6.1 Affine 계층

### 5.6.2 배치용 Affine 계층

### 5.6.3 Softmax-with-Loss 계층

## 5.7 오차역전파법 구현하기

### 5.7.1 신경망 학습의 전체 그림

### 5.7.2 오차역전파법을 적용한 신경망 구현하기

### 5.7.3 오차역전파법으로 구한 기울기 검증하기

### 5.7.4 오차역전파법을 사용한 학습 구현하기